Funny python functions in other files don't get refreshed unless restart kernel, below magic is witchcraft that kinda fixes problem.

In [2]:
%load_ext autoreload
%autoreload 2

# Custom script imports for compartmentalization 

In [1]:
import visualization, preprocess, metrics, naive_autoencoder, analyze_compression, wip_model

2025-04-15 23:08:19.925305: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-15 23:08:19.952684: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-15 23:08:19.952730: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-15 23:08:19.952807: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-15 23:08:19.960426: I tensorflow/core/platform/cpu_feature_g

# Check if Tensorflow sees your GPU and has CUDA
### Unnecessary unless you want to run large models on your GPU

In [8]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("Built with GPU support:", tf.test.is_built_with_gpu_support())
print("GPU available:", tf.config.list_physical_devices('GPU'))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2801297251558259844
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14211350528
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3413498496053723195
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 Ti SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]
Built with CUDA: True
Built with GPU support: True
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-04-15 23:11:31.311618: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-15 23:11:31.311677: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-15 23:11:31.311698: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-15 23:11:31.311850: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-15 23:11:31.311861: I tensorflow/core/co

# Autoencoder Model
### Ensure when creating new model that you return the autoencoder itself as well as latent space encoder.

In [3]:
sequence_length = 4
height = 64
width = 64
channels = 3

In [12]:
autoencoder, encoder = naive_autoencoder.build_conv_lstm_autoencoder(
    sequence_length=sequence_length,
    height=height,
    width=width,
    channels=channels
)

autoencoder.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 3, 64, 64, 3)]    0         
                                                                 
 time_distributed (TimeDistr  (None, 3, 64, 64, 32)    896       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 3, 64, 64, 64)    18496     
 tributed)                                                       
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 3, 64, 64, 64)     295168    
                                                                 
 time_distributed_2 (TimeDis  (None, 3, 64, 64, 64)    36928     
 tributed)                                                       
                                                             

In [11]:
import tensorflow as tf

# Define your input shape for the video sequences
input_shape = (sequence_length, height, width, channels)

# Build encoder and decoder
encoder = wip_model.build_encoder(input_shape=input_shape, latent_channels=32)
decoder = wip_model.build_decoder(latent_shape=encoder.output_shape[1:], output_channels=3)

# Vector quantization layer
vq_layer = wip_model.VectorQuantizer(num_embeddings=256, embedding_dim=32)

# Get latent shape without embedding dimension for PixelCNNPrior input
z_index_shape = encoder.output_shape[1:-1]  # (T, H, W)
print("PixelCNN input shape (T, H, W):", z_index_shape)

# Build PixelCNN prior
pixelcnn_prior = wip_model.PixelCNNPrior(input_shape=z_index_shape, num_embeddings=256)

# Compose the full VQ-VAE module
vqvae = wip_model.VQVAEModule(
    encoder=encoder,
    decoder=decoder,
    vq_layer=vq_layer,
    pixelcnn_prior=pixelcnn_prior,
    beta=1.0
)

# Compile with optimizer
vqvae.compile(optimizer=tf.keras.optimizers.Adam(1e-4))


PixelCNN input shape (T, H, W): (2, 16, 16)


In [12]:
import gc
import tensorflow as tf

# Clear TF session
tf.keras.backend.clear_session()

# Run garbage collection
gc.collect()


25089

# Preprocess
### Resizes, normalizes, and formats data-splitting into training and validation sets

In [13]:
import numpy as np

video_folder = "extracted_videos"

# Preprocess the videos
video_data, video_filenames = preprocess.preprocess_videos_with_mapping(
    video_folder=video_folder,
    sequence_length=sequence_length,
    height=height,
    width=width,
    channels=channels
)

# Split the data into training and validation sets
split_index = int(0.8 * len(video_data))
train_data = video_data[:split_index]
val_data = video_data[split_index:]

# Reshape the training and validation data to match the input shape of the autoencoder
train_data = train_data.reshape((-1, sequence_length, height, width, channels))
val_data = val_data.reshape((-1, sequence_length, height, width, channels))
train_data = train_data.astype(np.float32)
val_data = val_data.astype(np.float32)

print("Train data shape:", train_data.shape)  # Expected: (num_samples, sequence_length, height, width, channels)
print("Validation data shape:", val_data.shape)


Train data shape: (3128, 4, 64, 64, 3)
Validation data shape: (782, 4, 64, 64, 3)


In [14]:
# Preprocessing for WIP model, not necessary to run currently
import tensorflow as tf
import numpy as np

def make_dataset(array, batch_size=4, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices(array.astype(np.float32))
    if shuffle:
        ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_dataset(train_data, batch_size=8)
val_ds = make_dataset(val_data, batch_size=8, shuffle=False)
train_ds = tf.data.Dataset.from_tensor_slices(train_data).batch(8).prefetch(tf.data.AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices(val_data).batch(8).prefetch(tf.data.AUTOTUNE)



# ONLY RUN IF YOU'RE NATHAN OR WANT TO WASTE TIME

Or I suppose if you make a small baby model

In [39]:
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

# Output directory setup
run_name = "vqvae_video_compression"
output_dir = os.path.join("training_runs", run_name)
os.makedirs(output_dir, exist_ok=True)

# Callbacks
callbacks = [
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True),
    ModelCheckpoint(
    filepath=os.path.join(output_dir, "best_weights"),
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True
    ),
    TensorBoard(log_dir=os.path.join(output_dir, "logs"))
]

history = vqvae.fit(
    train_ds,
    validation_data=val_ds,
    epochs=7,
    callbacks=callbacks
)

Epoch 1/7
391/391 [==============================] - 19s 48ms/step - loss: 0.9669 - distortion: 0.4109 - rate: 0.5560 - val_loss: 1.0655 - val_distortion: 0.5410 - val_rate: 0.5245
Epoch 2/7
391/391 [==============================] - 18s 45ms/step - loss: 0.8106 - distortion: 0.3918 - rate: 0.4188 - val_loss: 1.2668 - val_distortion: 0.6076 - val_rate: 0.6592
Epoch 3/7
391/391 [==============================] - 18s 47ms/step - loss: 0.6695 - distortion: 0.3727 - rate: 0.2969 - val_loss: 0.9258 - val_distortion: 0.4512 - val_rate: 0.4747
Epoch 4/7
391/391 [==============================] - 18s 47ms/step - loss: 0.5867 - distortion: 0.3635 - rate: 0.2233 - val_loss: 0.7072 - val_distortion: 0.3852 - val_rate: 0.3220
Epoch 5/7
391/391 [==============================] - 18s 47ms/step - loss: 0.5258 - distortion: 0.3513 - rate: 0.1745 - val_loss: 0.5922 - val_distortion: 0.3306 - val_rate: 0.2616
Epoch 6/7
391/391 [==============================] - 18s 47ms/step - loss: 0.4692 - distortion:

In [ ]:
# Train the autoencoder
autoencoder.fit(
    train_data, train_data,
    validation_data=(val_data, val_data),
    epochs=10,
    batch_size=4
)

Train data shape: (37146, 3, 64, 64, 3)
Validation data shape: (9287, 3, 64, 64, 3)
Epoch 1/10


2025-04-03 13:52:01.078876: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2025-04-03 13:52:01.282855: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:231] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2025-04-03 13:52:01.282889: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Used ptxas at ptxas
2025-04-03 13:52:01.283137: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:317] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2025-04-03 13:52:01.815474: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fd9ee002930 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-04-03 13:52:01.815508: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecuto

9287/9287 [==============================] - 382s 41ms/step - loss: 7.7424e-04 - val_loss: 3.6359e-04
Epoch 2/10
9287/9287 [==============================] - 361s 39ms/step - loss: 1.4463e-04 - val_loss: 5.4172e-05
Epoch 3/10
9287/9287 [==============================] - 360s 39ms/step - loss: 9.0927e-05 - val_loss: 3.7964e-05
Epoch 4/10
9287/9287 [==============================] - 363s 39ms/step - loss: 7.0415e-05 - val_loss: 2.9867e-04
Epoch 5/10
9287/9287 [==============================] - 362s 39ms/step - loss: 6.1055e-05 - val_loss: 4.0588e-05
Epoch 6/10
9287/9287 [==============================] - 360s 39ms/step - loss: 5.1567e-05 - val_loss: 1.7583e-05
Epoch 7/10
9287/9287 [==============================] - 361s 39ms/step - loss: 4.6759e-05 - val_loss: 1.6843e-05
Epoch 8/10
9287/9287 [==============================] - 361s 39ms/step - loss: 4.2643e-05 - val_loss: 3.3355e-05
Epoch 9/10
9287/9287 [==============================] - 360s 39ms/step - loss: 3.9196e-05 - val_loss: 5.795

In [ ]:
# If have multiple autoencoders trained, can change name to load different one
encoder_file = "video_autoencoder.keras"

## SAVES ENCODER. ONLY USE IF YOU'VE JUST TRAINED A MODEL YOU WANT TO KEEP

In [ ]:
from tensorflow.keras.models import save_model

autoencoder.save(encoder_file)

In [15]:
import os

# Loads the wip model weights
weights_path = os.path.join("training_runs", "vqvae_video_compression", "best_weights")
vqvae.load_weights(weights_path)
print("✓ Weights loaded from:", weights_path)

✓ Weights loaded from: training_runs/vqvae_video_compression/best_weights


## LOADS MODEL, PROBABLY ALWAYS DO THIS

In [ ]:
from tensorflow.keras.models import load_model

autoencoder = load_model(encoder_file)

# It's the metrics, wtf else is there to say

# Can get metrics for multiple random videos based on split

In [9]:
metrics_val = metrics.evaluate_metrics_by_video(
    split="val",
    limit=3,
    video_data=video_data,
    video_filenames=video_filenames,
    autoencoder=vqvae,
    sequence_length=sequence_length,
    height=height,
    width=width
)
metrics_val

2025-04-15 23:00:25.602151: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2025-04-15 23:00:25.668451: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:231] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2025-04-15 23:00:25.668478: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Used ptxas at ptxas
2025-04-15 23:00:25.668514: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-04-15 23:00:25.751164: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:322] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2025-04-15 23:00:25.820554: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_k

32/99 [========>.....................] - ETA: 0s

2025-04-15 23:00:26.090365: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.


99/99 [==============================] - 1s 5ms/step
📹 Evaluating: celebv_-_Bdf9C0SAU_8.mp4 — 99 sequences
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/nchanak/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nchanak/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/nchanak/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth


2025-04-15 23:00:27.268343: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-04-15 23:00:27.268467: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-04-15 23:00:27.268542: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-04-15 23:00:27.268625: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-04-15 23:00:27.269854: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-04-15

121/121 [==============================] - 1s 5ms/step
📹 Evaluating: celebv_-_Bdf9C0SAU_3.mp4 — 121 sequences
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/nchanak/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
94/94 [==============================] - 0s 5ms/step
📹 Evaluating: celebv_-_Bdf9C0SAU_9.mp4 — 94 sequences
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/nchanak/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth


{'celebv_-_Bdf9C0SAU_8.mp4': {'psnr': 26.858852,
  'ssim': 0.6907404,
  'lpips': 0.23208558992153466},
 'celebv_-_Bdf9C0SAU_3.mp4': {'psnr': 27.105642,
  'ssim': 0.71718764,
  'lpips': 0.2303910284680276},
 'celebv_-_Bdf9C0SAU_9.mp4': {'psnr': 25.978357,
  'ssim': 0.7215823,
  'lpips': 0.22211066344158448}}

# Compression
### Yes very bad compression rn cuz latent space huge

In [42]:
import analyze_compression

# Assume you've already run
# autoencoder, encoder = build_conv_lstm_autoencoder(...)
# autoencoder.load_weights(...)
# video_data, video_filenames = preprocess_videos_with_mapping(...)

result = analyze_compression.analyze_compression_from_video_file(
    video_name="celebv_-_Bdf9C0SAU_0.mp4",
    autoencoder=autoencoder,
    encoder=encoder,
    sequence_length=3,
    height=64,
    width=64,
    cleanup=True
)


[▶] Processing 83 sequences from: celebv_-_Bdf9C0SAU_0.mp4
83/83 [==============================] - 0s 5ms/step

Compression Results for 'celebv_-_Bdf9C0SAU_0.mp4':
Raw 64x64 input size:    438.18 KB
Latent compressed size:  151012.24 KB
Estimated compression:   0.00x
Deleted: output_latent.npz
Deleted: output_original_input.npz
Deleted: output_reconstructed.npz


In [31]:
import importlib
import analyze_compression

importlib.reload(analyze_compression)
with tf.device('/CPU:0'):
    compression_results = analyze_compression.evaluate_compression_by_video(
        split="val",
        limit=3,
        video_data=video_data,
        video_filenames=video_filenames,
        autoencoder=vqvae,
        encoder=encoder,
        sequence_length=sequence_length,
        height=height,
        width=width
    )
compression_results

Evaluating compression on 3 val videos...

94/94 [==============================] - 2s 26ms/step


{'celebv_-_Bdf9C0SAU_8.mp4': {'video': 'celebv_-_Bdf9C0SAU_8.mp4',
  'num_sequences': 99,
  'num_frames': 396,
  'input_kb_estimated': 3282.0314071120415,
  'codec_kb': 36.412109375,
  'latent_kb_estimated': 13.292874336242676,
  'compression_ratio_codec': 90.13571208718368,
  'compression_ratio_latent': 246.90155974495812,
  'vq_time_secs': 3.647674322128296},
 'celebv_-_Bdf9C0SAU_3.mp4': {'video': 'celebv_-_Bdf9C0SAU_3.mp4',
  'num_sequences': 267,
  'num_frames': 1068,
  'input_kb_estimated': 9001.780255015248,
  'codec_kb': 134.998046875,
  'latent_kb_estimated': 34.84114074707031,
  'compression_ratio_codec': 66.68081845176879,
  'compression_ratio_latent': 258.3664042565019,
  'vq_time_secs': 8.400297164916992},
 'celebv_-_Bdf9C0SAU_9.mp4': {'video': 'celebv_-_Bdf9C0SAU_9.mp4',
  'num_sequences': 94,
  'num_frames': 376,
  'input_kb_estimated': 3259.8387960985624,
  'codec_kb': 31.8603515625,
  'latent_kb_estimated': 12.690022468566895,
  'compression_ratio_codec': 102.3164728645

requires: conda install -c conda-forge ffmpeg

alternatively: sudo apt install ffmpeg


In [10]:
play_video = "celebv_-_Bdf9C0SAU_6.mp4"

visualization.generate_and_display_comparison(
    video_name=play_video,
    autoencoder=vqvae,
    sequence_length=sequence_length,
    height=height,
    width=width,
    cleanup=True 
)

3/3 [==============================] - 1s 187ms/step
[✓] Comparison saved: comparison_celebv_-_Bdf9C0SAU_6.avi
[▶] Converting comparison_celebv_-_Bdf9C0SAU_6.avi → comparisons/comparison_celebv_-_Bdf9C0SAU_6.mp4
Deleted: comparison_celebv_-_Bdf9C0SAU_6.avi


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab